In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
package_path = '/kaggle/input/timmmodelss'#'/kaggle/input/pytorch-timm/pytorch_timm_models'
import sys;
sys.path.append(package_path)
import timm

In [ ]:
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
import torch
from torch import nn
import os
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset
from torch.cuda.amp import autocast, GradScaler
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2
from sklearn.metrics import log_loss

In [ ]:
CFG = {
    'fold_num': 5,
    'seed': 719,
    'model_arch_res101': 'resnet101d_320',#'tf_efficientnet_b4_ns',
    'model_arch_resnext101': 'ig_resnext101_32x8d',
    'model_arch_vit': 'vit_base_patch16_384',
    'model_arch_effb3': 'tf_efficientnet_b3_ns',
    'model_arch_dense': 'densenet201',
    'img_size': 512,
    'size_vit': 384,
    'epochs': 10,
    'train_bs': 16,
    'valid_bs': 16,
    'lr': 1e-4,
    'num_workers': 4,
    'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0',
    'tta': 4,
    'used_epochs_vit': [9],
    'used_epochs_res101': [9],
    'used_epochs_resnext': [9],
    'used_epochs_effb3': [9],
    'used_epochs_dense': [9],
    'weights_vit': [1],
    'weights_deit': [1],
    'weights_effb3': [1],
    'weights_dense': [1],
    'weights': [1],
    'weights_resnext': [1]
}

In [ ]:
fold_dir = '/kaggle/input/cassava-leaf-disease-classification/'
train = pd.read_csv(fold_dir+'train.csv')
submission = pd.read_csv(fold_dir+'sample_submission.csv')

vit_fold3 = '../input/vit384-fold0-smoothing/vit_base_patch16_384_fold_0_9.dms'
vit_path = [vit_fold3]

resnext = '../input/resnext101-512-fold-0/ig_resnext101_32x8d_fold_0_9'

resnext_path = [resnext]


effb3 = '../input/eff-b3-512-16/tf_efficientnet_b3_ns_fold_0_9'

effb3_path = [effb3]


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb
def rand_bbox(size, lam):
    W = size[0]
    H = size[1]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

In [ ]:
class CassavaDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        
        img = get_img(path)
        if self.transforms:
            img = self.transforms(image=img)['image']

        if self.output_label:
            return img, target
        else:
            return img
    

In [ ]:
def get_train_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_train_transforms_vit():
    return Compose([
            RandomResizedCrop(CFG['size_vit'], CFG['size_vit']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_valid_transforms():
    return Compose([
            CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_inference_transforms():
    return Compose([
            #CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_inference_transforms_vit():
    return Compose([
            #CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
            RandomResizedCrop(CFG['size_vit'], CFG['size_vit']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [ ]:
def prepare_dataloader(df, trn_idx, val_idx, data_root='train_images/'):
    
    train_ = df.loc[trn_idx,:].reset_index(drop=True)
    valid_ = df.loc[val_idx,:].reset_index(drop=True)
        
    train_ds = CassavaDataset(train_, data_root, transforms=get_train_transforms(), output_label=True, one_hot_label=False)
    valid_ds = CassavaDataset(valid_, data_root, transforms=get_valid_transforms(), output_label=True)
    train_ds_vit = CassavaDataset(train_, data_root, transforms=get_train_transforms_vit(), output_label=True, one_hot_label=False)
    
    
    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=CFG['train_bs'],
        pin_memory=False,
        drop_last=False,
        shuffle=True,        
        num_workers=CFG['num_workers'],
    )
    val_loader = torch.utils.data.DataLoader(
        valid_ds, 
        batch_size=CFG['valid_bs'],
        num_workers=CFG['num_workers'],
        shuffle=False,
        pin_memory=False,
    )
    return train_loader, val_loader

In [ ]:
def valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False):
    model.eval()

    loss_sum = 0
    sample_num = 0
    image_preds_all = []
    image_targets_all = []
    
    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    
    classacc = np.zeros(5)
    total = np.zeros(5)
    
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
        image_targets_all += [image_labels.detach().cpu().numpy()]
        
        loss = loss_fn(image_preds, image_labels)
        
        loss_sum += loss.item()*image_labels.shape[0]
        sample_num += image_labels.shape[0]  

        if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(val_loader)):
            description = f'epoch {epoch} loss: {loss_sum/sample_num:.4f}'
            pbar.set_description(description)
    
    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    print('validation multi-class accuracy = {:.4f}'.format((image_preds_all==image_targets_all).mean()))
    
    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum/sample_num)
        else:
            scheduler.step()

In [ ]:
def get_ground_truth(val_loader):

    image_targets_all = []
    
    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        
        image_targets_all += [image_labels.detach().cpu().numpy()]
        
    image_targets_all = np.concatenate(image_targets_all)
    return image_targets_all

In [ ]:
from collections import OrderedDict, Counter

class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        self.model.reset_classifier(num_classes=n_class)
#         n_features = self.model.head.in_features
#         self.model.head = nn.Linear(n_features, n_class)
    def forward(self, x):
        x = self.model(x)
        return x
    
class MyCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean'):
        super().__init__(weight=weight, reduction=reduction)
        self.weight = weight
        self.reduction = reduction

    def forward(self, inputs, targets):
        lsm = F.log_softmax(inputs, -1)

        if self.weight is not None:
            lsm = lsm * self.weight.unsqueeze(0)

        loss = -(targets * lsm).sum(-1)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [ ]:
def inference_one_epoch(model, data_loader, device, ResNet_Snap=False):
    model.eval()
    image_preds_all = []
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
#         image_preds = model(imgs)
        if not ResNet_Snap:
            image_preds = model(imgs)
        else:
            image_preds = model(imgs)[0]   
        
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
    image_preds_all = np.concatenate(image_preds_all, axis=0)

    return image_preds_all

In [ ]:
if __name__ == '__main__':
     # for training only, need nightly build pytorch

    seed_everything(CFG['seed'])
    
    folds = StratifiedKFold(n_splits=CFG['fold_num']).split(np.arange(train.shape[0]), train.label.values)
    
    for fold, (trn_idx, val_idx) in enumerate(folds):
        if fold > 0:
            break 

        print('Inference fold {} started'.format(fold))

        valid_ = train.loc[val_idx,:].reset_index(drop=True)
        valid_ds = CassavaDataset(valid_, fold_dir+'train_images/', transforms=get_inference_transforms(), output_label=True)
        
        test = pd.DataFrame()
        test['image_id'] = list(os.listdir(fold_dir+'test_images/'))
        test_ds = CassavaDataset(test, fold_dir+'test_images/', transforms=get_inference_transforms(), output_label=False)
        
        val_ds_vit = CassavaDataset(valid_, fold_dir+'train_images/', transforms=get_inference_transforms_vit(), output_label=True)
        
        val_loader = torch.utils.data.DataLoader(
            valid_ds, 
            batch_size=CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False,
        )
        
#         tst_loader = torch.utils.data.DataLoader(
#             test_ds, 
#             batch_size=CFG['valid_bs'],
#             num_workers=CFG['num_workers'],
#             shuffle=False,
#             pin_memory=False,
#         )
        
        val_loader_vit = torch.utils.data.DataLoader(
            val_ds_vit, 
            batch_size=CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False,
        )

        device = torch.device(CFG['device'])
        
        resnextmodel = CassvaImgClassifier(CFG['model_arch_resnext101'], train.label.nunique()).to(device)
        vitmodel = CassvaImgClassifier(CFG['model_arch_vit'], train.label.nunique()).to(device)
        effb3model = CassvaImgClassifier(CFG['model_arch_effb3'], train.label.nunique()).to(device)
        
        val_preds = []
        res101_preds = []
        resnext_preds = []
        vit_preds = []
        effb3_preds = []
        dense_preds = []

        
        
        for i, path in enumerate(vit_path): #enumerate(CFG['used_epochs_vit']):
            vitmodel.load_state_dict(torch.load(path))
            preds = []
            with torch.no_grad():
                for _ in range(CFG['tta']):
                    vit_preds += [CFG['weights_vit'][i]/sum(CFG['weights_vit'])/CFG['tta']*inference_one_epoch(vitmodel, val_loader_vit, device, ResNet_Snap=False)]
        vit_preds = np.mean(vit_preds, axis=0)
        del vitmodel

        
        for i, path in enumerate(effb3_path):
            effb3model.load_state_dict(torch.load(path))
            preds = []
            with torch.no_grad():
                for _ in range(CFG['tta']):
                    effb3_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(effb3model, val_loader, device, ResNet_Snap=False)]

        effb3_preds = np.mean(effb3_preds, axis=0)
        del effb3model
        
        
        
        for i, path in enumerate(resnext_path):
            resnextmodel.load_state_dict(torch.load(path))
            preds = []
            with torch.no_grad():
                for _ in range(CFG['tta']):
                    resnext_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(resnextmodel, val_loader, device, ResNet_Snap=False)]
                    
        resnext_preds = np.mean(resnext_preds, axis=0)
        del resnextmodel
        
        
        preds = vit_preds + effb3_preds + resnext_preds #0.3 * resnext_preds + 0.3 * effb3_preds
        label = np.argmax(preds,axis=1)
        print(label)
        
        image_targets_all = get_ground_truth(val_loader)
        print(image_targets_all)
        
        classacc = np.zeros(5)
        total = np.zeros(5)
        for i in range(0,len(image_targets_all)):
            total[image_targets_all[i]] += 1
            if label[i] == image_targets_all[i]:
                classacc[image_targets_all[i]] += 1

        print(classacc)
        print(total)
        print(classacc/total)
        print('validation multi-class accuracy = {:.4f}'.format((label==image_targets_all).mean()))
        
        for i in range(0,len(image_preds_all)):
            total[image_targets_all[i]] += 1
            if image_preds_all[i] == image_targets_all[i]:
                classacc[image_targets_all[i]] += 1

        print(classacc)
        print(total)
        print(classacc/total)

        torch.cuda.empty_cache()
        
        # for submission to leadboard
#         test['label'] = label#np.argmax(tst_preds, axis=1)
#         test.to_csv('submission.csv', index=False)
        print('end')
